In [1]:
import pandas as pd

In [2]:
# 加载star数据
from clickhouse_driver import Client

host = "localhost"
port = 9099
user = "sjjc"
password = "taosan506"
database = "dm"
send_receive_timeout = 10
client = Client(host=host, port=port, user=user, password=password, database=database,
                send_receive_timeout=send_receive_timeout)
star_df_list_sql = """
select uid, star_level from dm.pri_star_info;
"""
star_df_list = client.execute(star_df_list_sql)
attr_list = ['uid', 'star_level']
star_df = pd.DataFrame(columns=attr_list, data=star_df_list)
star_df.set_index('uid', inplace=True)

In [3]:
# 加载特征数据
data1 = pd.read_csv('./result.csv',encoding='utf-8')

In [16]:
# 合并到打分表
data_all = pd.merge(star_df,data1,how='left',left_index=True, right_index = True,sort=False,copy=True)

In [17]:
# 进行空缺值补全，用0补全
data_all.fillna(0,inplace=True)
data_all['star_level'] = data_all['star_level'].apply(int)

In [18]:
for i in data_all.columns:
    print(i)

star_level
uid
djk_num_x
normal_djk_num_x
abnormal_djk_num_x
not_using_djk
djk_credit
djk_deposit
djk_over_draft
djk_dlay_amt
djk_bankacct_num
djk_bankacct_sum
djk_is_etc
djk_mode_online
djk_sum_dlay_mths
djkfq_total_amt
djkfq_total_num
djkfq_avg_mths
djkfq_sum_mth_instl
djkfq_remain_unpay_money
contract_mx_num
contract_mx_buss_amt
contract_mx_norm_bal
contract_mx_dlay_bal
contract_mx_dull_bal
contract_mx_frz_amt
contract_buy_car
contract_buy_house
contract_business
contract_consume
contract_buy_car_sum
contract_buy_car_time
contract_buy_car_term_month
contract_buy_house_sum
contract_buy_house_time
contract_buy_house_term_month
contract_business_sum
contract_business_time
contract_business_term_month
contract_consume_sum
contract_consume_time
contract_consume_term_month
djk_mx_count
djk_mx_sum
djk_mx_avg
djk_mx_max
djk_mx_expense_count
djk_mx_expense_sum
djk_mx_expense_avg
djk_mx_expense_max
djk_mx_income_count
djk_mx_income_sum
djk_mx_income_avg
djk_mx_income_max
djk_mx_real_tran_amt


In [19]:
# 提取有效特征
# attrs = [ 'star_level',
#     'base_max_actu_intr',
# 'base_max_frst_intr',
# 'dsf_mx_count',
# 'max_intr',
# 'base_count_loan_amt',
# 'abnormal_djk_num_y',
# 'abnormal_djk_num_x',
# 'max_pay_term',
# 'owed_int_times_90g',
# 'month_avg_huanx_count',
# 'overdue_times_90g',
# 'owed_int_times_90l',
# 'overdue_times_90l',
# 'base_bad_bal',
# 'djk_sum_dlay_mths',
# 'education',
# 'base_due_intr',
# 'base_delay_bal',
# 'contract_mx_dull_bal',
#          'contract_buy_house_time',
# 'djk_is_etc',
# 'dsf_mx_sum',
# 'base_max_book_acct_amt',
# 'tran_amt_huanb',
# 'month_avg_shop_count',
# 'djk_credit',
# 'has_car_etc',
# 'sjyh_tran_amt_count',
# 'tran_amt_duebill',
# 'normal_djk_num_y',
# 'normal_djk_num_x',
# 'djk_mode_online',
# 'dsf_mx_avg',
# 'contract_consume_sum',
# 'dsf_mx_max',
# 'month_avg_sdrq_amt',
# 'contract_mx_num',
# 'not_using_djk',
# 'djk_bankacct_num',
# 'contract_mx_buss_amt',
# 'offline_tran_duebill',
# 'month_avg_sbyb_amt',
# 'djk_num_x',
# 'djk_num_y',
# 'contract_business_sum',
# 'contract_business',
# 'contract_business_term_month',
# 'djk_bankacct_sum',
# 'sjyh_tran_amt_sum',
# 'contract_consume_term_month',
# 'cust_bal_sum',
# 'contract_consume_time',
# 'cust_fin_bal',
# 'contract_consume',
# 'month_avg_sbyb_count',
# 'tran_amt_gzdf',
# 'cust_cd_bal',
# 'month_avg_sdrq_count',
# 'marrige',
# 'tran_times_gzdf',
# 'cust_sa_bal',
# 'birthday',
# 'has_social_security_gzdf',
# 'cust_all_bal_sum'
#          ]
attrs = [
    'star_level',
    'djk_num_x',
    'djk_credit',
    'djk_bankacct_sum',
    'is_black',
    'career',
    'cust_bal_sum',
    'cust_all_bal_sum',
    'cust_sa_bal',
    'cust_fin_bal',
    'cust_cd_bal',
    'base_sumloan_amt',
    'base_sum_credit_amt',
    'base_all_bal',
    'base_bad_bal',
    'base_norm_bal',
    'sex',
    'birthday',
    'education',
    'marrige'
]
data_all = data_all.loc[:,attrs]

In [63]:
# 切分数据集
from sklearn.model_selection import train_test_split
known_data = data_all[ (data_all['star_level']!=-1)]
target_data = data_all[(data_all['star_level']==-1)]

In [64]:
x = known_data.iloc[:,1:]
y = known_data.iloc[:,0:1]['star_level'].tolist()

In [65]:
x_train, x_test, y_train,y_test = train_test_split(x, y, test_size=0.2)

In [66]:
for i in range(1,10):
    print(i,"训练集次数",y_train.count(i),"测试集次数",y_test.count(i))

1 训练集次数 103172 测试集次数 25755
2 训练集次数 35414 测试集次数 8837
3 训练集次数 30592 测试集次数 7695
4 训练集次数 8023 测试集次数 2005
5 训练集次数 5673 测试集次数 1412
6 训练集次数 2795 测试集次数 710
7 训练集次数 36 测试集次数 10
8 训练集次数 11 测试集次数 5
9 训练集次数 3 测试集次数 1


In [67]:
# 使用随机森林算法
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=1000,n_jobs=-1,max_leaf_nodes=70)
rf_clf.fit(x_train,y_train)

RandomForestClassifier(max_leaf_nodes=70, n_estimators=1000, n_jobs=-1)

In [60]:
def true_score(result, test):
    t = 0
    diff = 0
    for i in range(len(result)):
        if result[i]==test[i]:
            t = t+1
        else:
            diff = diff + abs(int(result[i])-int(test[i]))

    print("差距",diff/len(result))
    return t/len(result)

In [61]:
y_pred = rf_clf.predict(x_test)
print(true_score(y_test, y_pred))

差距 0.5899957609156422
0.6533276812208563


In [62]:
from sklearn.metrics import confusion_matrix
cm = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index = [3,4,5,6,7,8,9],
    columns = [3,4,5,6,7,8,9]
)
print(cm)

      3  4  5  6  7  8  9
3  7706  0  0  0  0  0  0
4  1961  0  0  0  0  0  0
5  1401  0  0  0  0  0  0
6   715  0  0  0  0  0  0
7    10  0  0  0  0  0  0
8     1  0  0  0  0  0  0
9     1  0  0  0  0  0  0


In [28]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(multi_class='multinomial',solver='newton-cg')

In [29]:
log_reg.fit(x_train, y_train)

LogisticRegression(multi_class='multinomial', solver='newton-cg')

In [30]:
predict_train = log_reg.predict(x_train)

In [31]:
y_pred = log_reg.predict(x_test)
print(true_score(y_test, y_pred))

差距 0.8491492569459401
0.5527029937540383


In [32]:
from sklearn.metrics import confusion_matrix
cm = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index = [1,2,3,4,5,6,7,8,9],
    columns = [1,2,3,4,5,6,7,8,9]
)
print(cm)

       1  2  3  4  5  6  7  8  9
1  25662  0  0  0  0  0  0  0  0
2   9006  0  0  0  0  0  0  0  0
3   7725  0  0  0  0  0  0  0  0
4   1917  0  0  0  0  0  0  0  0
5   1407  0  0  0  0  0  0  0  0
6    694  0  0  0  0  0  0  0  0
7     13  0  0  0  0  0  0  0  0
8      5  0  0  0  0  0  0  0  0
9      1  0  0  0  0  0  0  0  0


In [27]:
# knn
from sklearn.neighbors import KNeighborsClassifier
modelknn = KNeighborsClassifier()
modelknn.fit(x_train, y_train)

KNeighborsClassifier()

In [28]:
y_pred = modelknn.predict(x_test)
print(true_score(y_test, y_pred))

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import confusion_matrix
cm = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index = [1,2,3,4,5,6,7,8,9],
    columns = [1,2,3,4,5,6,7,8,9]
)
print(cm)

In [33]:
from sklearn.ensemble import GradientBoostingClassifier
modelgbdt = GradientBoostingClassifier()
modelgbdt.fit(x_train, y_train)

GradientBoostingClassifier()

In [34]:
y_pred = modelgbdt.predict(x_test)
print(true_score(y_test, y_pred))

差距 0.8491492569459401
0.5527029937540383


In [35]:
from sklearn.metrics import confusion_matrix
cm = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index = [1,2,3,4,5,6,7,8,9],
    columns = [1,2,3,4,5,6,7,8,9]
)
print(cm)

       1  2  3  4  5  6  7  8  9
1  25662  0  0  0  0  0  0  0  0
2   9006  0  0  0  0  0  0  0  0
3   7725  0  0  0  0  0  0  0  0
4   1917  0  0  0  0  0  0  0  0
5   1407  0  0  0  0  0  0  0  0
6    694  0  0  0  0  0  0  0  0
7     13  0  0  0  0  0  0  0  0
8      5  0  0  0  0  0  0  0  0
9      1  0  0  0  0  0  0  0  0
